# Language Modelling in Hangman

# Overview

You'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [21]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [22]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`.

In [23]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [24]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [47]:
import nltk
from nltk.corpus import brown
import numpy as np

nltk.download('brown')
np.random.seed(60)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###
corpus = brown.words()
# lower the word first
lower_corpus = [word.lower() for word in corpus]
# get the set of unique word types
unique_corpus = list(set(lower_corpus))
# words entirely comprised of alphabetic characters
alpha_corpus = [word for word in unique_corpus if word.isalpha()]
# random shuffle the list
np.random.shuffle(alpha_corpus)
test_set = alpha_corpus[:1000]
training_set = alpha_corpus[1000:]
###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

[nltk_data] Downloading package brown to /Users/wangzeyu/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [26]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [27]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [28]:
def test_guesser(guesser, test):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [29]:
import string
import random

def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###
    # a list of alphabetic characters from a-z
    alphabet_list = list(string.ascii_lowercase)
    while True:
        # randomly choose a character, if not in guessed characters, return the character
        random_character = random.choice(alphabet_list)
        if random_character not in guessed:   
            return random_character.strip()
    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser, test_set)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = dexedrine
Number of mistakes made by the random guesser = 19

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.726


<b>For your testing:</b>

In [30]:
assert(result > 10 and result < 20)

### Question 3

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method (use the second answer space). Note that it takes `unigram_counts` as an additional argument.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [31]:
unigram_counts = {}

###
# Your answer BEGINS HERE
###
alphabet_list = list(string.ascii_lowercase)
# iterate through all the words in the training set and increment the frequency
# of the character by 1 every time we see the character
for word in training_set:
    for letter in list(word):
        if letter in unigram_counts.keys():
            unigram_counts[letter] += 1
        else:
            unigram_counts[letter] = 1

# if there are characters in none of the words in the training set, give it a 0 frequency
for char in alphabet_list:
    if char not in unigram_counts.keys():
        unigram_counts[char] = 0

###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    # sort the dict based on the frequencies(prob) of characters
    sorted_unigram_counts = dict(sorted(unigram_counts.items(), key=lambda item: item[1], reverse=True))
    highest_prob_keys = list(sorted_unigram_counts.keys())
    index = 0
    # guess from the highest prob characters to the lowest, return if the current character not in guessed
    while index < 26:
        if highest_prob_keys[index] not in guessed:
            return highest_prob_keys[index]
        else:
            index += 1
    return False
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_guesser, test_set)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.394


<b>For your testing:</b>

In [32]:
assert(result > 5 and result < 15)

### Question 4

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [33]:
unigram_counts_by_length = {} # dict, length of words are keys, lists of frequency dicts are values

###
# Your answer BEGINS HERE
###
word_lengths = list(set([len(word) for word in training_set]))
# same logic as in question 3, except this case one more loop for different word length encountered
# the frequency dicts for each length of words are computed
for length in word_lengths:
    freq_dict = dict.fromkeys(alphabet_list, 0)
    for word in training_set:
        if len(word) == length:
            for char in list(word):
                freq_dict[char] += 1
    unigram_counts_by_length[length] = freq_dict

###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    # if encounter a word length didn't see in the training set, behave like question 3
    if len(mask) not in unigram_counts_by_length.keys():
        guess = unigram_guesser(mask, guessed, unigram_counts)
    else: # guess based on the corresponding frequency dict for this length seen in the training process
        unigram_dict = unigram_counts_by_length[len(mask)]
        sorted_unigram_counts = dict(sorted(unigram_dict.items(), key=lambda item: item[1], reverse=True))
        high_prob_keys = list(sorted_unigram_counts.keys())
        index = 0
        # guess from the highest prob characters to the lowest, return if the current character not in guessed
        while index < 26:
            if high_prob_keys[index] not in guessed:
                guess = high_prob_keys[index]
                break
            else:
                index += 1
    
    return guess
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser, test_set)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.337


<b>For your testing:</b>

In [34]:
assert(result > 5 and result < 15)

### Question 5

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Note**:
- When backing-off to the unigram language model, you **must use the vanilla unigram language model that you have built in Q3**. Do not use the length-based unigram language model, or any fancy variant, or you will lose marks.
- You should build a **standard bigram language model**; i.e. do not do anything complicated like a bidirectional bigram language model.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [35]:
bigram_counts = {}

###
# Your answer BEGINS HERE
###
copied_alphabet_list = list(string.ascii_lowercase)
copied_alphabet_list.append('.')
all_bigrams = [x + y for x in copied_alphabet_list for y in copied_alphabet_list]
# all the combination of bigrams from the alphabet, plus a '.' representing the start or the end of the word
# even though the end character isn't necessary here 
bigram_counts = dict.fromkeys(all_bigrams, 0)

# accumulate the bigram frequencies for each word
for word in training_set:
    word = "." + word + "."
    bigrams = [word[i:i+2] for i in range(len(word) - 1)]
    for bigram in bigrams:
        bigram_counts[bigram] += 1

### 
# Your answer ENDS HERE
###
    
# smoothing not considered in this question, could be the case where there are unseen biagrams
def bigram_guesser(mask, guessed, bigram_counts=bigram_counts): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    prob_dict = dict.fromkeys(alphabet_list, 0)
    for index, char in enumerate(mask):
        if char == '_':
            # case 1: blank at the beginning of the masked word; apply bigram model with the starting symbol
            if index - 1 == -1:
                for alphabet in alphabet_list:
                    bigram = "." + alphabet
                    # divided by the length of the training set as the starting symbol occurs the same times
                    # as the number of words in the corpus
                    prob_dict[alphabet] += bigram_counts[bigram] / float(len(training_set))
            # case 2: no left context for a certain blank, apply unigram model
            elif mask[index - 1] == "_":
                for alphabet in alphabet_list:
                    prob_dict[alphabet] += unigram_counts[alphabet] / float(sum(unigram_counts.values()))
            # case 3: left context exists at the middle of the word (not starting), apply normal bigram model
            else:
                for alphabet in alphabet_list:
                    bigram = mask[index - 1] + alphabet
                    prob_dict[alphabet] += bigram_counts[bigram] / float(unigram_counts[mask[index - 1]])
    # same logic as the previous questions, give guesses based on the probabilities
    sorted_prob_dict = dict(sorted(prob_dict.items(), key=lambda item: item[1], reverse=True))
    possible_chars = list(sorted_prob_dict.keys())
    index = 0
    while index < 26:
        if possible_chars[index] not in guessed:
            return possible_chars[index]
        else:
            index += 1
    return False
    ###
    # Your answer ENDS HERE
    ###



result = test_guesser(bigram_guesser, test_set)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.59


### Question 6 

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

Note:
* When testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its _average_ performance, as there will be some variance to its performance depending on the training/test split.
* Your code must run under 2 minutes on Codalab; program that runs longer than that will be terminated and you will score 0.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [48]:
# initilise the frequency dict
trigram_counts = {}
all_trigrams = [x + y + z for x in copied_alphabet_list for y in copied_alphabet_list for z in copied_alphabet_list]
trigram_counts = dict.fromkeys(all_trigrams, 0)

# accumulate the trigram frequencies for each word
for word in training_set:
    word = ".." + word + ".."
    trigrams = [word[i:i+3] for i in range(len(word) - 2)]
    for trigram in trigrams:
        trigram_counts[trigram] += 1

In [37]:
def get_prob_addk(numerator, denominator, k):
    numerator += k
    denominator += k*27
    return numerator / float(denominator)

In [38]:
# this function is used to do trigram modelling(likelihood based on left context) of the masked word
def trigram_guesser(mask):
    prob_dict = dict.fromkeys(alphabet_list, 0)
    for index, char in enumerate(mask):
        if char == '_':
            # case 1: first character is blank, apply trigram
            if index == 0:
                for alphabet in alphabet_list:
                    trigram = ".." + alphabet
                    prob_dict[alphabet] += get_prob_addk(trigram_counts[trigram], len(training_set), 0.05)
            elif index == 1:
                if mask[0] != '_': # case 2: ".a_", apply trigram
                    for alphabet in alphabet_list:
                        trigram = "." + mask[0] + alphabet
                        prob_dict[alphabet] += get_prob_addk(trigram_counts[trigram], bigram_counts["." + mask[0]], 0.05)
                else: # case 3: ".__", apply unigram, no left context
                    for alphabet in alphabet_list:
                        prob_dict[alphabet] += get_prob_addk(unigram_counts[alphabet], sum(unigram_counts.values()), 0.05)
            else:
                if mask[index - 1] != '_' and mask[index - 2] != '_': # case 4: "aa_", trigram
                    for alphabet in alphabet_list:
                        trigram = mask[index - 2] + mask[index - 1] + alphabet
                        prob_dict[alphabet] += get_prob_addk(trigram_counts[trigram], bigram_counts[mask[index - 2] + mask[index - 1]], 0.05)
                elif mask[index - 1] != '_' and mask[index - 2] == '_': # case 5: "_a_", biagram as only on left context
                    for alphabet in alphabet_list:
                        bigram = mask[index - 1] + alphabet
                        prob_dict[alphabet] += get_prob_addk(bigram_counts[bigram], unigram_counts[mask[index - 1]], 0.05)
                else: # case 6: "a__" or "___", both can only apply unigram
                    for alphabet in alphabet_list:
                        prob_dict[alphabet] += get_prob_addk(unigram_counts[alphabet], sum(unigram_counts.values()), 0.05)
    return prob_dict

In [39]:
# this function is used to do trigram modelling in reverse direction(likelihood based on right context) of the masked word
def trigram_guesser_reverse(mask):
    prob_dict = dict.fromkeys(alphabet_list, 0)
    reverse_mask = list(reversed(mask))
    for index, char in enumerate(reverse_mask):
        if char == '_':
            # case 1: the last character is blank, apply trigram
            if index == 0: # note that this is actually the last character in mask
                for alphabet in alphabet_list:
                    trigram = alphabet + ".."
                    prob_dict[alphabet] += get_prob_addk(trigram_counts[trigram], len(training_set), 0.05)
            elif index == 1: 
                if reverse_mask[0] == '_': # case 2: "__.", apply unigram here since no right context
                    for alphabet in alphabet_list:
                        prob_dict[alphabet] += get_prob_addk(unigram_counts[alphabet], sum(unigram_counts.values()), 0.05)
                else: # case 3: "_a.", apply biagram here since only 1 right context is given
                    for alphabet in alphabet_list:
                        trigram = alphabet + reverse_mask[0] + "."
                        prob_dict[alphabet] += get_prob_addk(trigram_counts[trigram], bigram_counts[reverse_mask[0] + "."], 0.05)
            else:
                # case 4: "_aa", trigram, all the right contexts are given
                if reverse_mask[index - 1] != '_' and reverse_mask[index - 2] != '_':
                    for alphabet in alphabet_list:
                        trigram = alphabet + reverse_mask[index - 1] + reverse_mask[index - 2]
                        prob_dict[alphabet] += get_prob_addk(trigram_counts[trigram], bigram_counts[reverse_mask[index - 1] + reverse_mask[index - 2]], 0.05)
                # case 5: "_a_", bigram, 1 right context given
                elif reverse_mask[index - 1] != '_' and reverse_mask[index - 2] == '_':
                    for alphabet in alphabet_list:
                        bigram = alphabet + reverse_mask[index - 1]
                        prob_dict[alphabet] += get_prob_addk(bigram_counts[bigram], unigram_counts[reverse_mask[index - 1]], 0.05)
                else: # case 6: "__a" or "___", no neighbour right context (blank), unigram only
                    for alphabet in alphabet_list:
                        prob_dict[alphabet] += get_prob_addk(unigram_counts[alphabet], sum(unigram_counts.values()), 0.05)
    return prob_dict

In [49]:
###
# Your answer BEGINS HERE
###

# recurrent nn could definitely do better, if have time, will implement that
def my_amazing_ai_guesser(mask, guessed):
    # bidirectional trigram language model, combine the results (left context and right context) 
    # together to make the guess
    prob_dict1 = trigram_guesser(mask)
    prob_dict2 = trigram_guesser_reverse(mask)
    combined_dict = {key: prob_dict1[key] + prob_dict2[key] for key in prob_dict1}
    # sort the dict so that guess the alphabet from the highest prob to the lowest prob
    sorted_prob_dict = dict(sorted(combined_dict.items(), key=lambda item: item[1], reverse=True))
    possible_chars = list(sorted_prob_dict.keys())
    index = 0
    while index < 26:
        # if not in guessed, that would be the answer
        if possible_chars[index] not in guessed:
            return possible_chars[index]
        else:
            index += 1
    return False

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser, test_set)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.402


### Question 7

**Instructions:** Explain your approach and discuss your result below.

##### Your answer BEGINS HERE
The AI guesser is based on bidirectional trigram language model, with computing a prob_dict for alphabets from left context, and a prob_dict for alphabets from right context. Simle add k smoothing is used to deal with unseen grams in the training set, no fancier smoothing methods are used as the performance is already good enough. 10 different seeds are used to test the average performance of the language model and the average number of incorrect guesses for model trained under 10 different seeds is 7.4774. It's also worth noting that no neural network is implemented, so the running time is really fast and will not exeed the time limit.


##### Your answer ENDS HERE